In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
import numpy as np

from rich import print as pprint
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from quantumion.analog.operator import PauliX, PauliY, PauliZ
from quantumion.analog.circuit import AnalogCircuit
from quantumion.analog.gate import AnalogGate
from backends.analog.qutip import QutipBackend
from backends.task import Task, TaskArgsAnalog

from backends.provider import Provider

In [ ]:
ex = AnalogCircuit()
gate = AnalogGate(
    duration=1.0,
    unitary=[np.pi * PauliX],
    dissipation=[]
)
ex.add(gate=gate)
pprint(ex)

In [ ]:
json_str = ex.model_dump()
pprint(json_str)

ex_parse = AnalogCircuit(**json_str)

In [ ]:
args = TaskArgsAnalog(
    n_shots=100,
    fock_trunc=4,
    observables={'z': PauliZ, 'x': PauliX, 'y': PauliY},
    dt=0.1
)

task = Task(program=ex, args=args)

In [ ]:
backend = QutipBackend()
result = backend.run(task)
pprint(result)

In [ ]:
fig, ax = plt.subplots()
for name, expect in result.expect.items():
    ax.plot(result.times, expect, label=name)
ax.legend()
plt.show()

rho = np.outer(result.state, np.conj(result.state))
fig, axs = plt.subplots(1, 2)
sns.heatmap(rho.real, ax=axs[0])
sns.heatmap(rho.imag, ax=axs[1])
for ax in axs:
    ax.set_aspect('equal')
plt.show()

In [ ]:
provider = Provider()
job = provider.submit(task)
print(job)

In [ ]:
print(job)
results = provider.get_result(job)
pprint(results)

In [ ]:
from backends.task import Task, TaskArgsDigital
from backends.digital.data import TaskResultDigital
from backends.digital.tc import TensorCircuitBackend

from quantumion.digital.circuit import DigitalCircuit
from quantumion.digital.gate import Gate, H, CNOT
from quantumion.digital.statement import Statement, Measure, Barrier
from quantumion.digital.register import QuantumRegister, ClassicalRegister

qreg = QuantumRegister(id='q', reg=2)
creg = ClassicalRegister(id='c', reg=2)

circ = DigitalCircuit(qreg=qreg, creg=creg)
circ.add(H(qreg=qreg[0]))
circ.add(CNOT(qreg=qreg[0:2]))
pprint(circ)

In [ ]:
args = TaskArgsDigital(n_shots=100)
task = Task(program=circ, args=args)

backend = TensorCircuitBackend()
result = backend.run(task)
pprint(result)

In [ ]:
fig, ax = plt.subplots()
x = list(result.counts.keys())
ax.bar(x=x, height=list(result.counts.values()))
ax.set(xticks=x)
fig.show()

rho = np.outer(result.state, np.conj(result.state))
fig, axs = plt.subplots(1, 2)
sns.heatmap(rho.real, ax=axs[0])
sns.heatmap(rho.imag, ax=axs[1])
for ax in axs:
    ax.set_aspect('equal')
fig.show()